In [ ]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -q tiny-imagenet-200.zip

In [ ]:
import os
import shutil

val_dir = "tiny-imagenet-200/val"
images_dir = os.path.join(val_dir, "images")
annot_file = os.path.join(val_dir, "val_annotations.txt")

# Read annotations
with open(annot_file, "r") as f:
    lines = f.readlines()

# Create class folders and move images
for line in lines:
    parts = line.split("\t")
    file, class_name = parts[0], parts[1]
    class_dir = os.path.join(val_dir, class_name)
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)
    src = os.path.join(images_dir, file)
    dst = os.path.join(class_dir, file)
    if os.path.exists(src):
        shutil.move(src, dst)

# Remove leftover folder
shutil.rmtree(images_dir)

print("Validation set reorganized.")


In [ ]:
from torchvision import transforms

train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])


In [ ]:
from torchvision import datasets

train_dir = "tiny-imagenet-200/train"
val_dir   = "tiny-imagenet-200/val"

train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
test_dataset  = datasets.ImageFolder(val_dir, transform=test_transform)


In [ ]:
from torch.utils.data import DataLoader

batch_size = 128  # if OOM, reduce to 64

trainloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

testloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print("Train images:", len(train_dataset))
print("Test images:", len(test_dataset))


In [ ]:
data, labels = next(iter(trainloader))
print("Batch tensor shape:", data.shape)
print("Labels shape:", labels.shape)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4)
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, stride=1,padding=2)
        self.conv3 = nn.Conv2d(256,384,kernel_size=3,stride=1,padding=1)
        self.conv4 = nn.Conv2d(384,384,3,1,padding=1)
        self.conv5 = nn.Conv2d(384,256,3,1,padding=1)
        # Fully connected layers
        self.fc1 = nn.Linear(6 * 6 * 256, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 200)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x,3,2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 3,2)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.max_pool2d(x, 3,2)
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
model = AlexNet()

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
def train_one_epoch(epoch_index,training_loader):
    running_loss = 0.
    last_loss = 0.
    
    for i, data in enumerate(training_loader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            running_loss = 0.

    return last_loss

In [ ]:
# Initializing in a separate cell so we can easily add more epochs to the same run
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number,trainloader)


    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(testloader):
            vinputs, vlabels = vdata
            voutputs = model(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))



    epoch_number += 1